In [32]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import sys

## Initialising Bayesian Network Model

In [ ]:
## ORDER MATTERS!!
model = DiscreteBayesianNetwork([('Burglary', 'Alarm'), ('Earthquake', 'Alarm'), ('Alarm','John'), ('Alarm','Mary')])

# P(Burglary)
cpd_burglary = TabularCPD(variable='Burglary', variable_card=2, values=[[0.6], [0.4]])

# P(Earthquake)
cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2, values=[[0.998], [0.002]])

# P(Alarm | Burglary, Earthquake)
cpd_Alarm = TabularCPD(variable='Alarm',variable_card=2, values =[
                                        [0.99, 0.71, 0.06, 0.05],  # P(Alarm=0)
                                        [0.01, 0.29, 0.94, 0.95]   # P(Alarm=1)
                                    ],
                                  evidence = ['Burglary', 'Earthquake'],
                                  evidence_card=[2, 2])

# P(John | Alarm)
cpd_John = TabularCPD(variable='John', variable_card=2, values = [
                                    [0.95, 0.1],  # P(John=0)
                                    [0.05, 0.9]], # P(John=1)
                                    evidence = ['Alarm'],
                                    evidence_card=[2])

# P(Mary | Alarm)
cpd_Mary = TabularCPD(variable='Mary', variable_card=2, values = [
                                    [0.99, 0.3], # P(Mary=0)
                                    [0.01, 0.7]], # P(Mary=1)
                                    evidence = ['Alarm'],
                                    evidence_card=[2])

model.add_cpds(cpd_burglary, cpd_earthquake, cpd_Alarm, cpd_John, cpd_Mary)
print("Model is valid:", model.check_model())

Model is valid: True


In [31]:
print(model.get_cpds('Alarm'))

+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.99          | 0.71          | 0.06          | 0.05          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.01          | 0.29          | 0.94          | 0.95          |
+------------+---------------+---------------+---------------+---------------+


## Inference with Bayesian Network Model

In [33]:
# more efficient inference algorithm
infer = VariableElimination(model)

In [35]:
# Marginal probability of Burglary & Earthquake
phi_query = infer.query(['Burglary','Earthquake'],joint=True)
print(phi_query)

+-------------+---------------+----------------------------+
| Burglary    | Earthquake    |   phi(Burglary,Earthquake) |
+=============+===============+============================+
| Burglary(0) | Earthquake(0) |                     0.5988 |
+-------------+---------------+----------------------------+
| Burglary(0) | Earthquake(1) |                     0.0012 |
+-------------+---------------+----------------------------+
| Burglary(1) | Earthquake(0) |                     0.3992 |
+-------------+---------------+----------------------------+
| Burglary(1) | Earthquake(1) |                     0.0008 |
+-------------+---------------+----------------------------+


In [43]:
# Probability of Burglary given Mary calls
phi_query = infer.query(['Burglary'],evidence={'Mary':1},joint=True)
print(phi_query)


+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.0379 |
+-------------+-----------------+
| Burglary(1) |          0.9621 |
+-------------+-----------------+


In [44]:
# Probability of Burglary given ONLY Mary Calls
phi_query = infer.query(['Burglary'],evidence={'Mary':1,'John':0},joint=True)
print(phi_query)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.1864 |
+-------------+-----------------+
| Burglary(1) |          0.8136 |
+-------------+-----------------+


In [45]:
# Probability of Earthquake given Mary and John calls
phi_query = infer.query(['Earthquake'],evidence={'Mary':1,'John':1},joint=True)
print(phi_query)

+---------------+-------------------+
| Earthquake    |   phi(Earthquake) |
+===============+===================+
| Earthquake(0) |            0.9971 |
+---------------+-------------------+
| Earthquake(1) |            0.0029 |
+---------------+-------------------+
